# LINCS L1000 significant dysregulation

In [1]:
import pandas
import scipy.stats
import numpy
from statsmodels.sandbox.stats import multicomp

%matplotlib inline

In [2]:
# Read gene info
gene_df = pandas.read_table('data/consensi/genes.tsv', dtype={'entrez_gene_id': str})
gene_df.head(2)

,entrez_gene_id,status,symbol,type_of_gene,description
0,100,imputed,ADA,protein-coding,adenosine deaminase
1,1000,imputed,CDH2,protein-coding,"cadherin 2, type 1, N-cadherin (neuronal)"


In [3]:
def process_matrix_df(z_matrix_df):
    """
    Take a perturbagen by gene dataframe and extract signficantly dysregulated pairs.
    Returns `signif_df` with a row per dysregulated pair and `summary_df` which counts the number
    of dysregulated genes per perturbation.
    """
    melt_df = pandas.melt(z_matrix_df.reset_index(), id_vars='perturbagen', var_name = 'entrez_gene_id', value_name = 'z_score')
    melt_df = melt_df.merge(gene_df[['entrez_gene_id', 'symbol', 'status']])
    signif_df = melt_df.groupby(['perturbagen', 'status']).apply(get_significance).reset_index(drop=True)
    signif_df = signif_df.sort_values(['perturbagen', 'symbol'])
    summary_df = signif_df.groupby(['perturbagen', 'direction', 'status']).apply(lambda df: pandas.Series({'count': len(df)})).reset_index()
    summary_df = summary_df.pivot_table('count', 'perturbagen', ['direction', 'status']).fillna(0).astype(int).reset_index()
    summary_df.columns = ['-'.join(col).strip('-') for col in summary_df.columns.values]
    return signif_df, summary_df

In [4]:
def get_significance(df):
    """
    Get signficant perturbagen-gene pairs using a Bonferroni adjustment.
    Different modes for measured and imputed genes.
    """
    p_values = 2 * scipy.stats.norm.cdf(-df.z_score.abs())
    if all(df.status == 'measured'):
        alpha = 0.05
        max_diffex = len(df)
    elif all(df.status == 'imputed'):
        alpha = 0.05
        max_diffex = 1000
    else:
        raise ValueError('Invalid status')
    reject, pvals_corrected, alpha_c_sidak, alpha_c_bonf = multicomp.multipletests(p_values, alpha=alpha, method='bonferroni')
    df['direction'] = df.z_score.map(lambda x: 'up' if x > 0 else 'down')
    df['nlog10_bonferroni_pval'] = -numpy.log10(pvals_corrected)
    df = df[reject]
    df = df.sort_values('nlog10_bonferroni_pval', ascending=False).iloc[:max_diffex, :]
    return df

In [5]:
# Iterate through all consensi and process them
for pert_kind in ['drugbank', 'knockdown', 'overexpression', 'pert_id']:
    print(pert_kind)
    path = 'data/consensi/consensi-{}.tsv.bz2'.format(pert_kind)
    z_matrix_df = pandas.read_table(path, index_col=0)
    signif_df, summary_df = process_matrix_df(z_matrix_df)
    path = 'data/consensi/signif/dysreg-{}.tsv'.format(pert_kind)
    signif_df.to_csv(path, index=False, sep='\t', float_format='%.3f')
    path = 'data/consensi/signif/dysreg-{}-summary.tsv'.format(pert_kind)
    summary_df.to_csv(path, index=False, sep='\t')

drugbank
knockdown
overexpression
pert_id


In [6]:
signif_df.head()

,perturbagen,entrez_gene_id,z_score,symbol,status,direction,nlog10_bonferroni_pval
0,56582,3303,5.542,HSPA1A,measured,up,4.533937
3,5981,10551,5.871,AGR2,imputed,up,4.551160
2,5981,440,-7.248,ASNS,imputed,down,8.561512
11,5981,637,6.852,BID,measured,up,8.147383
7,5981,873,8.856,CBR1,measured,up,15.090913


In [7]:
summary_df.head()

,perturbagen,down-imputed,down-measured,up-imputed,up-measured
0,56582,0,0,0,1
1,5981,2,5,2,36
2,7150,2,1,11,5
3,ABL1_G2A,1,3,1,0
4,ABL1_T315I,2,4,1,5


In [8]:
# compress `dysreg-pert_id.tsv` to reduce file size
! gzip data/consensi/signif/dysreg-pert_id.tsv